In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

def clean_and_normalize_ML(df, columns_to_exclude=[]):
    numerical_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    non_numerical_features = df.select_dtypes(exclude=['int64', 'float64']).columns.tolist()

    numerical_features_to_normalize = [col for col in numerical_features if col not in columns_to_exclude]

    numerical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_pipeline, numerical_features_to_normalize)
        ],
        remainder='passthrough'
    )

    df_cleaned = preprocessor.fit_transform(df)

    feature_names = numerical_features_to_normalize + non_numerical_features + columns_to_exclude

    df_cleaned = pd.DataFrame(df_cleaned, columns=feature_names)

    return df_cleaned

df = pd.read_csv('energy.csv')
columns_to_exclude = ['Year', 'Population']
df_cleaned = clean_and_normalize(df, columns_to_exclude=columns_to_exclude)
df_cleaned.to_csv('energy_cleaned.csv', index=False)


In [ ]:
import pandas as pd

def normalize_to_3nf(df):
    df.dropna(inplace=True)

    df.drop_duplicates(inplace=True)


    normalized_tables = {}

    for col in df.select_dtypes(include=['object']).columns:
        normalized_tables[col] = pd.DataFrame(df[col].unique(), columns=[col])
        normalized_tables[col]['id'] = normalized_tables[col].index + 1

    for col, table in normalized_tables.items():
        df = df.merge(table, on=col, how='left')
        df.drop(columns=[col], inplace=True)
        df.rename(columns={'id': f'{col}_id'}, inplace=True)


    id_columns = [col for col in df.columns if col.endswith('_id')]
    other_columns = [col for col in df.columns if not col.endswith('_id')]
    df = df[id_columns + other_columns]

    normalized_tables['fact_table'] = df

    return normalized_tables

df = pd.read_csv('energy.csv')
normalized_tables = normalize_to_3nf(df)

for name, table in normalized_tables.items():
    table.to_csv(f'tnf{name}.csv', index=False)

In [ ]:
import pandas as pd
import sqlite3

def csv_to_sqlite(csv_file, sqlite_db, table_name):
    df = pd.read_csv(csv_file)

    conn = sqlite3.connect(sqlite_db)

    df.to_sql(table_name, conn, if_exists='replace', index=False)

    conn.close()
    print(f"Data from {csv_file} has been successfully written to {table_name} table in {sqlite_db}.")

csv_file = 'temp.csv'
sqlite_db = 'climate.db'
table_name = 'temperatures'

csv_to_sqlite(csv_file, sqlite_db, table_name)

Data from temp.csv has been successfully written to temperatures table in climate.db.


In [ ]:
system_role = '''Write python code to select relevant data and draw the chart. Use the SQL query provided to connect to the Database and retrieve data. The database name is chinook.db. Please create a data frame from relevant data and print the dataframe. Only use the sql i provide and do not generate your own. create a function called return_df() to return the df. '''

In [ ]:
question_original = "Find the percentage change in average temperature per country"
max_tokens = 2500